In [1]:
library('reticulate')
boto3 <- import('boto3')
athena <- boto3$client('athena', region_name='us-east-1')

In [15]:
query <- "SELECT label, value FROM cookbook_athena_db.athena_table;"
database <- "cookbook_athena_db"
results_bucket <- "s3://cookbook-athena-results"

In [11]:
execute_athena_query <- function(query, database, results_bucket) {
    response <- athena$start_query_execution(
        QueryString = query,
        QueryExecutionContext = dict(Database=database),
        ResultConfiguration = dict(OutputLocation=results_bucket)
    )    
    
    return(response$QueryExecutionId)
}

In [3]:
get_output_path <- function(execution_id) {
    query_details <- athena$get_query_execution(
        QueryExecutionId = execution_id
    )
    
    execution <- query_details$QueryExecution
    configuration <- execution$ResultConfiguration
    return(configuration$OutputLocation)
}

In [4]:
cmd <- function(bash_command) {
    print(bash_command)
    output <- system(bash_command, intern=TRUE)
    last_line = ""
    
    for (line in output) { 
        cat(line)
        cat("\n")
        last_line = line 
    }
    
    return(last_line) 
}

In [7]:
cmd("mkdir -p tmp")

[1] "mkdir -p tmp"


[1] ""

In [16]:
execution_id <- execute_athena_query(query, database, results_bucket)
output_path <- get_output_path(execution_id)
output_path

[1] "s3://cookbook-athena-results/987d6fbe-6773-4279-a904-a351d070599b.csv"

In [18]:
command <- paste("aws s3 cp", output_path, "tmp/output.csv")

cmd(command)

[1] "aws s3 cp s3://cookbook-athena-results/987d6fbe-6773-4279-a904-a351d070599b.csv tmp/output.csv"
download: s3://cookbook-athena-results/987d6fbe-6773-4279-a904-a351d070599b.csv to tmp/output.csv


[1] "Completed 16.8 KiB/16.8 KiB (359.1 KiB/s) with 1 file(s) remaining\rdownload: s3://cookbook-athena-results/987d6fbe-6773-4279-a904-a351d070599b.csv to tmp/output.csv"

In [19]:
df <- read.csv("tmp/output.csv", header=TRUE)
df

label,value
<chr>,<int>
BVWIGECENF,2
EMELOEGMJV,1
RJIVHCZUBL,5
WVLWTZZGGN,8
YXFAVHXUBY,3
ZMLZMGCYGG,4
XJPTWNPRWU,2
YQEATBSHCI,1
XRRNFFLGOG,1


In [21]:
query <- "
USING EXTERNAL FUNCTION detect_anomaly(value INT)
    RETURNS DOUBLE
    SAGEMAKER 'sagemaker-cookbook-rcf'
SELECT label, value, detect_anomaly(value) AS anomaly_score
    FROM cookbook_athena_db.athena_table"

In [22]:
execution_id <- execute_athena_query(query, database, results_bucket)
output_path <- get_output_path(execution_id)
output_path

[1] "s3://cookbook-athena-results/303e123b-2e39-41d2-999b-6420dabf3b03.csv"

In [24]:
command <- paste("aws s3 cp", output_path, "tmp/output.csv")
cmd(command)

[1] "aws s3 cp s3://cookbook-athena-results/303e123b-2e39-41d2-999b-6420dabf3b03.csv tmp/output.csv"
download: s3://cookbook-athena-results/303e123b-2e39-41d2-999b-6420dabf3b03.csv to tmp/output.csv


[1] "Completed 37.0 KiB/37.0 KiB (728.3 KiB/s) with 1 file(s) remaining\rdownload: s3://cookbook-athena-results/303e123b-2e39-41d2-999b-6420dabf3b03.csv to tmp/output.csv"

In [25]:
df <- read.csv("tmp/output.csv", header=TRUE)
df

label,value,anomaly_score
<chr>,<int>,<dbl>
SIRTDXCEZH,3,0.9313710
UARRYGLNRB,4,0.8485596
ORFPUJWEFC,10,1.3119031
XUGMJRDURG,73,2.5832871
EFKZBCPNXH,6,0.8280757
LQEHDUVUUA,5,0.8484080
ZZZAPLVJFJ,3,0.9313710
DHZIQWAGQT,1,1.2045547
SBCPVTAWNZ,7,0.8708918


In [32]:
subset(df, anomaly_score > 2)

,label,value,anomaly_score
,<chr>,<int>,<dbl>
4,XUGMJRDURG,73,2.583287
13,LLZTRLHDPL,70,2.528207
18,FKYFVHEPMH,78,2.574346
21,CKVJEZWBUN,76,2.603616
26,FFOAWOWXQY,80,2.637032
36,HKTZQNIVNN,72,2.584881
61,TEEONPEITR,74,2.600842
67,DBRHTMQCUX,71,2.544879
76,HOSFZMJSBN,77,2.610782
